## Load necessary Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import nltk
#nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/emotion-detection-from-text/tweet_emotions.csv


## Load the datasets

In [2]:
df = pd.read_csv('/kaggle/input/emotion-detection-from-text/tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [4]:
df['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [5]:
#Let us take the top 3 categories and leave out the rest.
shortlist = ['neutral', "happiness", "worry"]
df_subset = df[df['sentiment'].isin(shortlist)]
df_subset.shape

(22306, 3)

## Text pre-processing:
Tweets are different. Somethings to consider:
- Removing @mentions, and urls perhaps?
- using NLTK Tweet tokenizer instead of a regular one
- stopwords, numbers as usual.

In [6]:
#strip_handles removes personal information such as twitter handles, which don't
#contribute to emotion in the tweet. preserve_case=False converts everything to lowercase.
tweeter = TweetTokenizer(strip_handles=True,preserve_case=False)
mystopwords = set(stopwords.words("english"))

#Function to tokenize tweets, remove stopwords and numbers. 
#Keeping punctuations and emoticon symbols could be relevant for this task!
def preprocess_corpus(texts):
    def remove_stops_digits(tokens):
        #Nested function that removes stopwords and digits from a list of tokens
        return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    #This return statement below uses the above function to process twitter tokenizer output further. 
    return [remove_stops_digits(tweeter.tokenize(content)) for content in texts]

#df_subset contains only the three categories we chose. 
mydata = preprocess_corpus(df_subset['content'])
mycats = df_subset['sentiment']
print(len(mydata), len(mycats))

22306 22306


### Split data into train and test, following the usual process

In [7]:
train_data, test_data, train_cats, test_cats = train_test_split(mydata,mycats,random_state=2, stratify = mycats)

### Prepare training data in doc2vec format:

In [8]:
#prepare training data in doc2vec format:
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(train_data)]
train_doc2vec[:5]

[TaggedDocument(words=['listening', 'simply', 'awesome', 'ratatat', 'bank', 'holiday', 'monday', 'bbq', 'later', 'http://bit.ly/gJqSh'], tags=['0']),
 TaggedDocument(words=['sister', 'said', 'yayyy', 'hannah', 'hoedown', 'today', 'haaaa', 'hannah', 'hoe', 'apparently', 'hoe', '.'], tags=['1']),
 TaggedDocument(words=['bad', 'feeling', 'time', 'next', 'yr', ',', 'fav', 'musical', 'may', 'longer', 'west', 'end'], tags=['2']),
 TaggedDocument(words=['oh', 'yeah', ',', 'came', 'back', 'weekend', 'cosplay', 'photo', 'marathons'], tags=['3']),
 TaggedDocument(words=['happy', 'mothers', 'day', '!', '!', '!'], tags=['4'])]

## Build the model

#### `models.doc2vec` [Doc2vec paragraph embeddings](https://radimrehurek.com/gensim/models/doc2vec.html)

In [9]:
#Train a doc2vec model to learn tweet representations. Use only training data!!
D2V = Doc2Vec(vector_size=50, alpha=0.025, min_count=5, dm =1, epochs=100) # `dm=1`, 'distributed memory' 
D2V.build_vocab(train_doc2vec)

print('Count of documents =',D2V.corpus_count)   # Count of documents
print('Epochs =',D2V.epochs)     #Epoch
print('vector_size =',D2V.vector_size)        # Dimensionality of the feature vectors
print('Window =',D2V.window)  # The maximum distance between the current and predicted word within a sentence

Count of documents = 16729
Epochs = 100
vector_size = 50
Window = 5


In [10]:
D2V.train(train_doc2vec, total_examples=D2V.corpus_count, epochs=D2V.epochs)

### Save & Load the Doc2Vec model

In [11]:
D2V.save("d2v.model")
print("Model Saved")

Model Saved


In [12]:
#Infer the feature representation for training and test data using the trained model
model= Doc2Vec.load("d2v.model")

In [13]:
#infer in multiple steps to get a stable representation. 
train_vectors =  [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in train_data]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in test_data]

### Precict the class with Logistic Regression Model

In [14]:
#Use any regular classifier like logistic regression
from sklearn.linear_model import LogisticRegression

myclass = LogisticRegression(class_weight="balanced") #because classes are not balanced. 
myclass.fit(train_vectors, train_cats)

LogisticRegression(class_weight='balanced')

In [15]:
preds = myclass.predict(test_vectors)
preds[:5]

array(['neutral', 'worry', 'worry', 'worry', 'worry'], dtype=object)

## Classification Report

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_cats, preds))

              precision    recall  f1-score   support

   happiness       0.45      0.52      0.48      1302
     neutral       0.48      0.56      0.52      2160
       worry       0.56      0.41      0.47      2115

    accuracy                           0.50      5577
   macro avg       0.50      0.50      0.49      5577
weighted avg       0.50      0.50      0.49      5577

